In [1]:
from osgeo import gdal, gdalconst
import numpy as np
from os import listdir
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [2]:
driver = gdal.GetDriverByName("HFA")
driver.Register()

6

# load feature files from folder

#load data by pixel

In [3]:
def load_data_pixel(filepath):
    filename_list = listdir(filepath)
    f = []
    for filename in filename_list:
        if filename.endswith(".dat"):
            f.append(filepath+"\\"+filename)
    dsbypixel = []
    for feature_file in f:
        ds = gdal.Open(feature_file)
        cols = ds.RasterXSize
        rows = ds.RasterYSize
        bands = ds.RasterCount
        for i in range(0, rows, 1):
            for j in range(0, cols, 1):
                data = ds.ReadAsArray(j, i,1,1).reshape(4,)/10000
                dsbypixel.append(data)
    return dsbypixel

In [4]:
h = ["Shrub", "Masson's pine","Other broadleaf","Bare soil","Chinese fir"]
dsbypixel_s = []
pixel_labels = []
j = 1
for i in h:
    a = load_data_pixel(r"G:\Paper\python\TensorFlow\test04\dataset"+"\\"+i)
    dsbypixel_s.extend(a)
    b = np.ones([len(a),])*j
    pixel_labels = np.hstack((pixel_labels,b))
    j = j+1
    print(i+" nums: "+ str(len(a)))
print(len(dsbypixel_s))
print(pixel_labels, pixel_labels.shape)
    

Shrub nums: 2178
Masson's pine nums: 2669
Other broadleaf nums: 2413
Bare soil nums: 2228
Chinese fir nums: 3686
13174
[ 1.  1.  1. ...,  5.  5.  5.] (13174,)


# one hot for labels dataset

labels for pixel datasets

In [5]:
onehot_encoder = OneHotEncoder(sparse=False)
pixel_labels_onehot = onehot_encoder.fit_transform(pixel_labels.reshape(-1, 1))
print(pixel_labels_onehot)
print(pixel_labels_onehot.shape)

[[ 1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]]
(13174, 5)


# make datasets

In [6]:
pixel_dataset = {}
pixel_dataset['feature'] = np.array(dsbypixel_s)
pixel_dataset['target'] = pixel_labels_onehot
pixel_dataset['target name'] = h

In [7]:
print(pixel_dataset)

{'feature': array([[ 0.059 ,  0.1003,  0.0693,  0.4776],
       [ 0.058 ,  0.0988,  0.067 ,  0.4698],
       [ 0.0573,  0.0982,  0.0655,  0.4635],
       ..., 
       [ 0.0484,  0.0657,  0.056 ,  0.3388],
       [ 0.0495,  0.067 ,  0.0581,  0.3448],
       [ 0.0514,  0.0697,  0.0617,  0.3577]]), 'target': array([[ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.]]), 'target name': ['brush', 'maweisong', 'qitakuoye', 'road', 'shamu']}


In [8]:
x_train, x_validation, y_train, y_validation =train_test_split(pixel_dataset["feature"], pixel_dataset["target"],
                                                   random_state=0)

In [9]:
print(x_train[1],x_train.shape, y_train[1],y_train.shape)
print(x_validation[1],x_validation.shape, y_validation[1],y_validation.shape)

[ 0.0508  0.0847  0.0589  0.4909] (9880, 4) [ 1.  0.  0.  0.  0.] (9880, 5)
[ 0.0454  0.0707  0.0546  0.4112] (3294, 4) [ 0.  0.  1.  0.  0.] (3294, 5)


# Tensorflow for MLP

In [10]:
#load mnist dataset
# mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

# make batches
batch_size = 50
n_batch = x_train.shape[0] // batch_size
def variable_summaries(var):
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean)
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        tf.summary.histogram("histogram", var)
        
# make placeholder
with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 4],name="x")
    y = tf.placeholder(tf.float32, [None, 5],name="y")
# keep_prob define dropout parameter
    keep_prob = tf.placeholder(tf.float32) 

# neruon network framework
with tf.name_scope("hlayer1"):
    with tf.name_scope("w1"):
        w1 = tf.Variable(tf.truncated_normal([4,100], stddev=0.1))
        variable_summaries(w1)
    with tf.name_scope("b1"):
        b1 = tf.Variable(tf.zeros([100])+0.1)
        variable_summaries(b1)
    L1 = tf.nn.tanh(tf.matmul(x, w1)+b1)
    L1_dropout = tf.nn.dropout(L1, keep_prob)
with tf.name_scope("hlayer2"):
    with tf.name_scope("w2"):
        w2 = tf.Variable(tf.truncated_normal([100,100], stddev=0.1))
        variable_summaries(w2)
    with tf.name_scope("b2"):
        b2 = tf.Variable(tf.zeros([100])+0.1)
        variable_summaries(b2)
    L2 = tf.nn.relu(tf.matmul(L1_dropout, w2)+b2)
    L2_dropout = tf.nn.dropout(L2, keep_prob)
with tf.name_scope("sotfmax"):
    with tf.name_scope("w3"):
        w3 = tf.Variable(tf.truncated_normal([100,5], stddev=0.1))
        variable_summaries(w3)
    with tf.name_scope("b3"):
        b3 = tf.Variable(tf.zeros([5])+0.1)
        variable_summaries(b3)
    prediction = tf.nn.softmax(tf.matmul(L2_dropout, w3)+b3)

# cross-entropy
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
    tf.summary.scalar("loss",loss)
# Change the optimizer
with tf.name_scope("train"):  
    train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
# initial variables
init = tf.global_variables_initializer()
# accuracy
with tf.name_scope("accuracy"): 
    with tf.name_scope("correct_prediction"):  
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope("accuracy"): 
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy",accuracy)

saver = tf.train.Saver()
merge = tf.summary.merge_all()
train_acc_summary = tf.summary.scalar("train_accuracy",accuracy)
validation_acc_summary = tf.summary.scalar("validation", accuracy)
best_validation_accuracy = 0.0
last_improvement = 0
require_improvement = 100
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter(r"G:\Paper\python\TensorFlow\test06\logfile/mlp_lr==1e-3_100_kp==0.5", sess.graph)
    best_validation_accuracy = 0.0
    last_improvement = 0
    require_improvement = 100
    total_iterations = 0
    for epoch in range(1000):
        idxs = np.arange(x_train.shape[0])
        x_random = x_train[idxs]
        y_random = y_train[idxs]
        total_iterations += 1
        for i in range(n_batch):
            batch_x = x_train[i*batch_size:(i+1)*batch_size]
            batch_y = y_train[i*batch_size:(i+1)*batch_size]
            summary,_ = sess.run([merge,train_step], feed_dict={x:batch_x, y:batch_y, keep_prob:0.5})
        train_acc_str = sess.run(train_acc_summary, feed_dict={x:x_train, y:y_train, keep_prob:1.0})
        validation_acc_str = sess.run(validation_acc_summary,feed_dict={x:x_validation, y:y_validation, keep_prob:1.0})
        writer.add_summary(train_acc_str,epoch)
        writer.add_summary(validation_acc_str, epoch)
        writer.add_summary(summary,epoch)
        if (total_iterations % 10 == 0) or (epoch == 999):
            train_acc = sess.run(accuracy, feed_dict={x:x_train, y:y_train, keep_prob:1.0})
            validation_acc = sess.run(accuracy, feed_dict={x:x_validation, y:y_validation, keep_prob:1.0})
            if validation_acc > best_validation_accuracy:
                best_validation_accuracy = validation_acc
                last_improvement = total_iterations
                saver.save(sess,r"G:\Paper\python\TensorFlow\test06\net_mlp_lr==le-3_100_kp==0.5--/my_net.ckpt")
                improved_str = "*"
            else:
                improved_str = " "
            msg = "Iter: {0:>6}, Train Accuracy: {1:>6.1%}, Validation Acc: {2:>6.1%} {3}"
            print(msg.format(epoch + 1, train_acc, validation_acc, improved_str))
        if total_iterations - last_improvement > require_improvement:
            print("No improvement found in a while, stopping optimization")
            break


Iter:     10, Train Accuracy:  91.0%, Validation Acc:  91.9% *
Iter:     20, Train Accuracy:  93.0%, Validation Acc:  93.4% *
Iter:     30, Train Accuracy:  93.4%, Validation Acc:  93.8% *
Iter:     40, Train Accuracy:  93.8%, Validation Acc:  94.2% *
Iter:     50, Train Accuracy:  93.9%, Validation Acc:  94.4% *
Iter:     60, Train Accuracy:  94.3%, Validation Acc:  94.7% *
Iter:     70, Train Accuracy:  94.5%, Validation Acc:  95.0% *
Iter:     80, Train Accuracy:  94.7%, Validation Acc:  94.9%  
Iter:     90, Train Accuracy:  95.0%, Validation Acc:  95.2% *
Iter:    100, Train Accuracy:  94.8%, Validation Acc:  95.1%  
Iter:    110, Train Accuracy:  95.1%, Validation Acc:  95.2%  
Iter:    120, Train Accuracy:  95.1%, Validation Acc:  95.3% *
Iter:    130, Train Accuracy:  95.4%, Validation Acc:  95.6% *
Iter:    140, Train Accuracy:  95.5%, Validation Acc:  96.1% *
Iter:    150, Train Accuracy:  94.9%, Validation Acc:  95.4%  
Iter:    160, Train Accuracy:  95.9%, Validation Acc:  